In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

print("Training data")
train_df.head()

Training data


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# --- 1. Обработка 'Age' (Возраст) ---
# 'Безопасный' способ: мы присваиваем результат обратно колонке.
median_age = train_df['Age'].median() # Сначала найдем медиану
train_df['Age'] = train_df['Age'].fillna(median_age)
test_df['Age'] = test_df['Age'].fillna(median_age)

# --- 2. Обработка 'Embarked' (Порт посадки) ---
mode_embarked = train_df['Embarked'].mode()[0] # Найдем самое частое значение
train_df['Embarked'] = train_df['Embarked'].fillna(mode_embarked)
test_df['Embarked'] = test_df['Embarked'].fillna(mode_embarked)

# --- 3. Обработка 'Fare' (Цена билета) ---
median_fare = test_df['Fare'].median() # Найдем медиану цены
test_df['Fare'] = test_df['Fare'].fillna(median_fare)

# --- 4. Превращаем текст в числа ('Sex') ---
# Этот код был в порядке, но для единообразия напишем его так же
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})

# --- 5. Превращаем текст в числа ('Embarked') ---
# Этот код тоже был в порядке
train_df = pd.get_dummies(train_df, columns=['Embarked'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Embarked'], drop_first=True)

print("Данные после обработки (без предупреждений):")
train_df.head()

Данные после обработки (без предупреждений):


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,False,False
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,False,True
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,False,True


In [3]:
from sklearn.linear_model import LogisticRegression

# --- 1. Выбираем признаки (фичи) ---
# Это список колонок, которые, как мы думаем, влияют на выживание.
# Мы берем только те, что есть и в train, и в test, и которые являются числами.
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']
target = 'Survived' # Это то, что мы хотим предсказать

# Готовим данные для модели
X_train = train_df[features] # Признаки из train.csv
y_train = train_df[target]   # Ответы (0 или 1) из train.csv
X_test = test_df[features]   # Те же признаки из test.csv (ответов тут нет)

# --- 2. Создаем и обучаем модель ---
model = LogisticRegression(max_iter=1000) # max_iter=1000, чтобы модель точно обучилась
model.fit(X_train, y_train)

print("Модель обучена!")


Модель обучена!


In [4]:
# --- 1. Делаем предсказания ---
predictions = model.predict(X_test)

# --- 2. Готовим файл для отправки ---
# Kaggle требует файл в формате: две колонки (PassengerId и Survived)
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': predictions
})

# --- 3. Сохраняем файл ---
# Файл сохранится в ту же папку, где лежит твой ноутбук (т.е. в папку 'notebook')
submission.to_csv('submission_basic.csv', index=False)

print("Файл 'submission_basic.csv' создан в папке 'notebook'!")
submission.head()

Файл 'submission_basic.csv' создан в папке 'notebook'!


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
